### Описание проекта

Нужно защитить данные клиентов страховой компании «Хоть потоп».
Разработать такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обосновать корректность его работы.


Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

### Описание данных

 - Набор данных находится в файле /datasets/insurance.csv.
 - Признаки: пол, возраст и зарплата застрахованного, количество членов его семьи.
 - Целевой признак: количество страховых выплат клиенту за последние 5 лет.

Подключим необходимые библиотеки

In [2]:
import pandas as pd
import numpy as np

from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA

import plotly.express as px

## Шаг 1. Загрузим и посмотрим на данные

In [3]:
data = pd.read_csv('https://code.s3.yandex.net/datasets/insurance.csv')
data.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


Датасет содержит 5000 наблюдений и 5 признаков. Данные имеют формат int64 и float64, также в датасете отсутствуют пропуски.

In [5]:
data.describe()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,0.499000,30.952800,39916.360000,1.194200,0.148000
std,0.500049,8.440807,9900.083569,1.091387,0.463183
min,0.000000,18.000000,5300.000000,0.000000,0.000000
25%,0.000000,24.000000,33300.000000,0.000000,0.000000
50%,0.000000,30.000000,40200.000000,1.000000,0.000000
75%,1.000000,37.000000,46600.000000,2.000000,0.000000
max,1.000000,65.000000,79000.000000,6.000000,5.000000


Судя по описательной статистике данные не имеют ярковыраженных выбросов (mean ~ median).
Примерный портрет среднего клиента страховой компании:
- возраст 31 год +-8 лет
- зарплата 40к +- 10к
- 1-2 члена семьи

In [6]:
np.linalg.inv(np.random.random_sample(size=(5,5)))

array([[ 0.60412557, -1.2869503 ,  2.7632673 , -2.02030853,  0.21954331],
       [ 2.12119305, -1.48272201, -8.85654707,  0.26489642,  6.19105318],
       [ 0.36531461,  2.80229828, -1.38414823,  2.78586251, -3.1359106 ],
       [-2.71811624, -0.43951651, 11.05337636, -1.04367583, -4.74293325],
       [-1.07794365,  0.06999057,  3.73459158, -1.55894668, -0.36867899]])

In [10]:
def multiply_data_by_inv_matrix(data: pd.DataFrame) -> pd.DataFrame:
    count_data, count_columns = data.shape
    columns = data.columns
    inv_data = pd.DataFrame()
    inv_matrix = np.linalg.inv(np.random.random_sample(size=(count_columns, count_columns)))

    for index in range(0, count_data, count_columns):
        if index == 0:
            old_index = index
            continue
        sample = data.loc[old_index:index, :] @ inv_matrix
        inv_data = pd.concat(
            [inv_data, sample],
            axis=0
        )
        old_index = index

    inv_data.columns = columns
    return inv_data

In [11]:
inv_data = multiply_data_by_inv_matrix(data)

In [12]:
inv_data

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,-344694.971877,821850.819630,-245223.336841,-820093.068883,1.184720e+06
1,-264092.461574,629662.559402,-187875.636145,-628322.859404,9.077017e+05
2,-145975.921688,348032.879342,-103844.952133,-347292.022991,5.017222e+05
3,-289742.834137,690851.595250,-206138.175487,-689370.297999,9.958521e+05
4,-181407.199660,432515.403266,-129052.639514,-431591.921866,6.234960e+05
...,...,...,...,...,...
4991,-266134.365515,634557.064173,-189343.296238,-633193.242103,9.147060e+05
4992,-379383.993882,904574.416898,-269900.799012,-902649.376495,1.303967e+06
4993,-250195.434203,596532.463832,-177993.589645,-595255.616744,8.599213e+05
4994,-357132.754360,851531.311973,-254077.160453,-849711.079430,1.227474e+06


In [13]:
features = data.drop('Страховые выплаты', axis=1)
target = data['Страховые выплаты']

features_inv = inv_data.drop('Страховые выплаты', axis=1)
target_inv = inv_data['Страховые выплаты']

X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=.02, random_state=25)
X_train_inv, X_test_inv, y_train_inv, y_test_inv = train_test_split(features_inv, target_inv, test_size=.02, random_state=25)

In [14]:
model = LinearRegression()
model.fit(X_train, y_train)
pred = model.predict(X_test)
print(r2_score(y_test, pred))

0.2964817324283996


In [16]:
model = LinearRegression()
model.fit(X_train_inv, y_train_inv)
pred = model.predict(X_test_inv)
print(r2_score(y_test_inv, pred))

0.9999999999351529


In [36]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

s_scaler = StandardScaler()
minmax_scaler = MinMaxScaler()
X_train_s_scaled = s_scaler.fit_transform(X_train)
X_train_minmax_scaled = minmax_scaler.fit_transform(X_train)

model.fit(X_train_s_scaled, y_train)
pred = model.predict(s_scaler.fit_transform(X_test))
print(f"После стандартного скейлера: {r2_score(y_test, pred)}")

model.fit(X_train_minmax_scaled, y_train)
pred = model.predict(minmax_scaler.fit_transform(X_test))

print(f"После минмакс скейлера: {r2_score(y_test, pred)}")

После стандартного скейлера: 0.25756910603033767
После минмакс скейлера: -0.15590979525468085
